# Lastni tipi

Poleg bogatega nabora vgrajenih tipov si tipe v OCamlu lahko definiramo tudi sami.

## Zapisni tipi

### Za tipe si lahko definiramo **okrajšave**

Najenostavnejši način za definicijo tipov so okrajšave obstoječih tipov. Na primer, tip za $\mathbb{R}^3$ si lahko definiramo kot:

In [1]:
type r3 = float * float * float

type r3 = float * float * float


In [5]:
let vsota_r3 ((x1, y1, z1) : r3) ((x2, y2, z2) : r3) : r3 =
  (x1 +. x2, y1 +. y2, z1 +. z2)

val vsota_r3 : r3 -> r3 -> r3 = <fun>


In [6]:
let vsota_r3' : r3 -> r3 -> r3 =
 fun (x1, y1, z1) (x2, y2, z2) -> (x1 +. x2, y1 +. y2, z1 +. z2)

val vsota_r3' : r3 -> r3 -> r3 = <fun>


### **Zapisni tip** podamo z zahtevanimi polji

Recimo, da si definiramo datume s trojicami celih števil.

In [7]:
type datum = int * int * int

type datum = int * int * int


Kateri vrstni red smo uporabili: dan, mesec, leto, kot smo navajeni v Sloveniji, ali leto, mesec, dan, kot je mednarodni standard? Mogoče celo mesec, dan, leto, kot je navada v Združenih državah?

Zmešnjavi se lahko izognemo, če komponente poimenujemo. V OCamlu to storimo z zapisnimi tipi, ki jih podamo tako, da naštejemo imena polj ter njihove tipe:

In [1]:
type datum = { dan : int; mesec : int; leto : int }

type datum = { dan : int; mesec : int; leto : int; }


Vrednosti tipov pišemo podobno, le da jih podamo z `=`:

In [9]:
let osamosvojitev = { dan = 25; mesec = 6; leto = 1991 }

val osamosvojitev : datum = {dan = 25; mesec = 6; leto = 1991}


### Do polj lahko dostopamo prek **projekcij**

Do posameznih komponent dostopamo z `zapis.ime_polja`:

In [10]:
let je_prestopno leto =
  (leto mod 4 = 0 && leto mod 100 <> 0) || leto mod 400 = 0
  
let dolzina_meseca leto =
  function
  | 4 | 6 | 9 | 11 -> 30
  | 2 -> if je_prestopno leto then 29 else 28
  | _ -> 31

val je_prestopno : int -> bool = <fun>


val dolzina_meseca : int -> int -> int = <fun>


In [11]:
let je_veljaven datum =
  let veljaven_dan = 1 <= datum.dan && datum.dan <= dolzina_meseca datum.leto datum.mesec
  and veljaven_mesec = 1 <= datum.mesec && datum.mesec <= 12
  in
  veljaven_dan && veljaven_mesec

val je_veljaven : datum -> bool = <fun>


### Zapise lahko **razstavljamo** tudi prek vzorcev

In [12]:
let je_veljaven {dan = d; mesec = m; leto = l} =
  let veljaven_dan = 1 <= d && d <= dolzina_meseca l m
  and veljaven_mesec = 1 <= m && m <= 12
  in
  veljaven_dan && veljaven_mesec

val je_veljaven : datum -> bool = <fun>


In [13]:
let je_veljaven {dan; mesec; leto} =
  let veljaven_dan = 1 <= dan && dan <= dolzina_meseca leto mesec
  and veljaven_mesec = 1 <= mesec && mesec <= 12
  in
  veljaven_dan && veljaven_mesec

val je_veljaven : datum -> bool = <fun>


### Zapise lahko **posodabljamo** z `with`

Z zapisom `{zapis with polje1 = vrednost1, ...}` ustvarimo nov zapis, ki ima z izjemo naštetih vrednosti polja enaka prvotnemu:

In [14]:
let pred_sto_leti datum =
  {dan = datum.dan; mesec = datum.mesec; leto = datum.leto - 100}

val pred_sto_leti : datum -> datum = <fun>


In [15]:
let pred_sto_leti datum =
  {datum with leto = datum.leto - 100}

val pred_sto_leti : datum -> datum = <fun>


### Tipi nam pomagajo pri **preverjanju veljavnosti**

In [24]:
let naredi_veljaven_datum datum =
  if je_veljaven datum then Some datum else None

val naredi_veljaven_datum : datum -> datum option = <fun>


In [23]:
ustvari_datum 29 2 1900

- : datum option = None


## Naštevni tipi

Najzanimivejši tipi, ki jih lahko definiramo, so _naštevni tipi_. Tako kot pri zapisnih tipih bomo tudi vrednosti naštevnih tipov sestavljali iz manjših vrednosti. Razlika med njimi je v tem, da morajo biti pri zapisnih tipih prisotne vrednosti _vseh_ naštetih polj, mora biti pri naštevnih tipih prisotna _natanko ena_ izmed naštetih variant.

### **Naštevni tip** podamo z možnimi **variantami**

Naštevne tipe podamo tako, da naštejemo možne variante, od katerih je vsaka podana s svojim _konstruktorjem_.

Če želimo opisati pošiljko, jo opišemo s tremi vrednostmi: naslovnikom, naslovom in načinom dostave. Če želimo dostavo omejiti na dva možna načina, pa uporabimo naštevni tip z dvema variantama:

In [2]:
type dostava =
  | OsebniPrevzem
  | PoPosti
type posiljka = {
  naslovnik : string;
  naslov : string;
  dostava : dostava
}

type dostava = OsebniPrevzem | PoPosti


type posiljka = { naslovnik : string; naslov : string; dostava : dostava; }


### Funkcije na naštevnih tipih podamo **po kosih**

Tako kot vsote naštejemo po kosih, lahko prek `match` ali `function` po kosih tudi definiramo funkcije na njih.

In [27]:
type dostava =
  | OsebniPrevzem
  | PoPosti

type dostava = OsebniPrevzem | PoPosti


In [28]:
let cena_dostave =
  function
  | OsebniPrevzem -> 0.
  | PoPosti -> 2.5

val cena_dostave : dostava -> float = <fun>


### Prevajalnik nas sam opozori na **manjkajoče primere**

Če tip razširimo z dodatno varianto, nas bo prevajalnik sam opozoril nanjo:

In [5]:
type dostava =
  | OsebniPrevzem
  | PoPosti
  | HitraDostava

type dostava = OsebniPrevzem | PoPosti | HitraDostava


In [6]:
let cena_dostave =
  function
  | OsebniPrevzem -> 0.
  | PoPosti -> 2.5

File "[6]", lines 2-4, characters 2-18:
2 | ..function
3 |   | OsebniPrevzem -> 0.
4 |   | PoPosti -> 2.5
Warning 8 [partial-match]: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
HitraDostava


val cena_dostave : dostava -> float = <fun>


In [7]:
let cena_dostave =
  function
  | OsebniPrevzem -> 0.
  | PoPosti -> 2.5
  | HitraDostava -> 4.

val cena_dostave : dostava -> float = <fun>


### Konstruktorji lahko sprejmejo tudi **argumente**

Vsak izmed naštetih konstruktorjev lahko sprejme tudi argumente vnaprej določenega tipa:

In [35]:
type dostava =
  | OsebniPrevzem
  | PoPosti of string
  | HitraDostava of string
type posiljka = {
  naslovnik : string;
  dostava : dostava
}

type dostava = OsebniPrevzem | PoPosti of string | HitraDostava of string


type posiljka = { naslovnik : string; dostava : dostava; }


In [39]:
HitraDostava "Jadranska ulica 21, 1000 Ljubljana"

- : dostava = HitraDostava "Jadranska ulica 21, 1000 Ljubljana"


### Tipi z **eno varianto** so uporabni za ločevanje tipov

In [45]:
{ naslovnik = "Matija Pretnar";
  dostava = HitraDostava (
    Telefon "01 4766 600",
    Naslov "Jadranska 21"
  )}

error: compile_error

In [41]:
type naslov = Naslov of string
type telefon = Telefon of string
type dostava =
  | OsebniPrevzem
  | PoPosti of naslov
  | HitraDostava of naslov * telefon
type posiljka = {
  naslovnik : string;
  dostava : dostava
}

type naslov = Naslov of string


type telefon = Telefon of string


type dostava =
    OsebniPrevzem
  | PoPosti of naslov
  | HitraDostava of naslov * telefon


type posiljka = { naslovnik : string; dostava : dostava; }


### Naštevni tipi so lahko **parametrizirani** in **rekurzivni**

Tako kot na primer vgrajeni tip `list` lahko tudi naši tipi vsebujejo parametre:

```ocaml
type 'a option =
  | None
  | Some of 'a
```

Naštevni tipi so lahko definirani tudi rekurzivno. Primer takega tipa, ki ga že poznamo, so seznami. Vsak seznam je bodisi prazen, bodisi sestavljen iz glave in repa:

```ocaml
type 'a seznam =
  | Prazen
  | Sestavljen of 'a * 'a seznam
```

Sedaj tudi vidimo, zakaj `::` lahko uporabljamo v vzorcih - ni namreč običajna funkcija za sestavljanje seznamov, temveč konstruktor tipa seznamov.